In [18]:
import sys
sys.path.append('/work/attalos/karllab41-attalos/')
import argparse
import gzip
import numpy as np
import tensorflow as tf
from tensorflow.contrib import learn as learn
import matplotlib.pylab as plt

from attalos.dataset.dataset import Dataset
from attalos.dataset.onehot import OneHot

%matplotlib inline

In [2]:
xTr_file = '/data/features/image/mscoco_train2014_inception_uncompressed_0701.hdf5'
xTr_file = '/data/features/image/iaprtc12_train_inception.hdf5'
yTr_file = '/data/features/text/mscoco_train2014_text.json.gz'
yTr_file = '/data/features/text/iaprtc12_train_text.json.gz'

xTe_file = '/data/features/image/iaprtc12_test_inception.hdf5'
yTe_file = '/data/features/text/iaprtc12_test_text.json.gz'

train_dataset = Dataset(xTr_file, yTr_file)
test_dataset = Dataset(xTr_file, yTr_file)
onehot = OneHot([train_dataset, test_dataset])

# print onehot.data_mapping
# dataset_tags = set()
# for dataset in [train_dataset]:
#     for tags in dataset.text_feats.values():
#         for tag in tags:
#             dataset_tags.add(tag.replace(' ', '_'))

In [3]:
def train_dnn_regressor(image_feats, 
                        word_feats, 
                        regressor=None,
                        hidden_units=[200,200]):
    if regressor is None:
        regressor = learn.TensorFlowDNNRegressor(hidden_units=hidden_units, 
                                                 steps=10, 
                                                 continue_training=True,
                                                 verbose=0)
    regressor.fit(image_feats, word_feats)
    return regressor

In [9]:
from IPython.display import clear_output

regressor_function = train_dnn_regressor; 
batch_size=128
hidden_units=[200,200]
    
image_feats, text_tags = test_dataset.get_next_batch(batch_size)
for i in range(batch_size):
    image_feats[i, :] /= np.linalg.norm(image_feats[i, :])
val_text_tags = np.array(map(onehot.get_multiple, text_tags))

regressor = None
for epoch in range(20):
    for batch in range(int(train_dataset.num_images/batch_size)):
        image_feats, text_tags = train_dataset.get_next_batch(batch_size)
        for i in range(batch_size):
            image_feats[i, :] /= np.linalg.norm(image_feats[i, :])
        word_feats = np.array(map(onehot.get_multiple, text_tags))
        regressor = train_dnn_regressor(image_feats, word_feats, regressor, hidden_units=hidden_units)
        print("Epoch {}, BatchNo {}".format(epoch, batch))
        clear_output()

## Prepare a file to be written out to the word2vec code

### Reverse dictionary

In [ ]:
# Create reverse dictionary
fwddict = onehot.data_mapping
bckdict = ['onehot'] * onehot.vocab_size;
for dictkey in fwddict:
    bckdict[ fwddict[dictkey] ] = dictkey

### Weights of the final layer

In [115]:
# Get the final weights from the optimizer function
Wc = regressor.weights_[-1].T

# Normalize if desired
Wc /= np.linalg.norm(Wc,axis=1)[:,None]

In [119]:
regressor.bias_

[array([-0.07590414, -0.02508961, -0.02022664, -0.00137483, -0.16465868,
        -0.02889749, -0.04837446,  0.0169291 , -0.04138856, -0.0416892 ,
        -0.10086012, -0.11905735, -0.15623811,  0.04444357,  0.01670133,
        -0.02025465, -0.07446151, -0.05730958,  0.0828125 ,  0.08852769,
        -0.04758041, -0.03296917, -0.02279255, -0.05245261, -0.13308807,
        -0.03535907,  0.18981375, -0.04896884, -0.06028128, -0.04883436,
        -0.0557026 ,  0.04439161,  0.00105747,  0.03204271, -0.07604173,
        -0.04013797, -0.10636482, -0.08885053, -0.04063022, -0.13761815,
         0.02796038, -0.02378296, -0.05810253, -0.02739257, -0.03879549,
         0.00517381, -0.01674707, -0.02943473,  0.01427416, -0.13853873,
        -0.02020191,  0.01691185,  0.00501648,  0.02194239,  0.02207491,
        -0.1222863 , -0.06299523, -0.08270027, -0.07105076,  0.12674223,
         0.03545761, -0.02627793, -0.05560167, -0.02002595, -0.02702051,
        -0.00920878, -0.17041561, -0.22201359,  0.0

### Write the final weights to file

In [116]:
fp = open('imvecs_directnorm.txt','wb')
fp.write("%s %s\n" % (str(Wc.shape[0]), str(Wc.shape[1]))) 
for i in range(len(bckdict)):
    fp.write('%s ' % (str(bckdict[i])))
    for j in range(Wc.shape[1]):
        fp.write('%s ' %(str(Wc[i,j])))
    fp.write("\n")
fp.close()

## Evaluation

In [20]:
image_feats, text_tags = test_dataset.get_next_batch(test_dataset.num_images)
for i in range(batch_size):
    image_feats[i, :] /= np.linalg.norm(image_feats[i, :])
true_text_tags = np.array(map(onehot.get_multiple, text_tags))

text_pred = regressor.predict(image_feats)

In [26]:
from oct2py import octave
octave.addpath('../evaluation/')
[precision, recall, f1score] = octave.evaluate(true_text_tags.T, text_pred.T, 5)
print precision
print recall
print f1score

0.310790402936
0.200218828364
0.243541942224


### Check Yonas's Code

### Had to rewrite top $k$ to use evaluation matrix.

In [ ]:
def get_top_k(text_corr, reverse_dictionary, topk=5):
    
    ## My new code to fit the correlation matrix instead
    indices = text_corr.argsort()[-1:-(topk+1):-1]
    return [reverse_dictionary[i] for i in indices]

    mh = MinHeap(k)
    for word in w2v_lookup:
        mh.insert(distance_func(target_vector, w2v_lookup[word]), word)
    return mh.get_result()

In [87]:
def calculate_precision(ground_truth, predicted):
    ground_truth_set = set(ground_truth)
    predicted_set = set(predicted)
    return float(len(ground_truth_set.intersection(predicted_set)))/len(predicted_set)

def calculate_recall(ground_truth, predicted):
    ground_truth_set = set(ground_truth)
    predicted_set = set(predicted)
    return float(len(ground_truth_set.intersection(predicted_set)))/len(ground_truth_set)


def get_pr(ground_truths, predictions):
    precisions = []
    recalls = []
    f1s = []
    for i in range(len(ground_truths)):
        precision = calculate_precision(ground_truths[i], predictions[i])
        recall = calculate_recall(ground_truths[i], predictions[i])
 
        if precision == 0 and recall == 0:
            f1 = 0
        else:
            f1 = 2*precision*recall/(precision + recall)
 
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    return np.mean(precisions), np.mean(recalls),  np.mean(f1s)

def evaluate_regressor(regressor, val_image_feats, val_text_tags, w2v_model, k=5):
    """
    Takes a regressor and returns the precision/recall on the test data
    Args:
        regressor: a tensorflow.contrib.learn regression estimator
        val_image_feats: Image features to test performance on
        val_text_tags: Text Tags to test performance on
        w2v_model: a dictionary like object where the keys are words and the values are word vectors
        k: Top number of items to retrieve to test precision/recall on

    Returns:
        (precision, recall, f1): A tuple of precision, recall, and F1 score
    """
    val_pred = regressor.predict(val_image_feats)
    topks = []
    for i in range(val_pred.shape[0]):
        topk = get_top_k(val_pred[i, :]/np.linalg.norm(val_pred[i, :]), w2v_model, k)        
        topks.append(topk)

    return get_pr(val_text_tags, topks)


# Create reverse dictionary
fwddict = onehot.data_mapping
bckdict = ['onehot'] * onehot.vocab_size;
for dictkey in fwddict:
    bckdict[ fwddict[dictkey] ] = dictkey

evaluate_regressor(regressor, image_feats, text_tags, bckdict)

(0.42206623266345888, 0.38634658161851165, 0.38211745751893539)

In [41]:
regressor.weights_[2].shape
onehot.create_data_mapping?